## Rozpoznawanie samochodów w czasie rzeczywistym z użyciem sieci neuronowych

---

W tym etapie wykorzystam własny zbiór danych

Użyty model : https://github.com/ultralytics/yolov5

---

### Zacznę od wczytania potrzebnych bibliotek

In [ ]:
import torch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
!pip install ultralytics
from ultralytics import YOLO
import PIL
import shutil
from google.colab import userdata


Klonuje repozytorium yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17483, done.
remote: Total 17483 (delta 0), reused 0 (delta 0), pack-reused 17483 (from 1)
Receiving objects: 100% (17483/17483), 16.53 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (11988/11988), done.


Pobieram własny zbiór danych

In [ ]:
os.environ["GITHUB_TOKEN"] = userdata.get('GITHUB_TOKEN')


In [ ]:
!git clone https://$GITHUB_TOKEN@github.com/iochgsthsu/car_dataset.git

Cloning into 'car_dataset'...
remote: Enumerating objects: 576, done.
remote: Total 576 (delta 0), reused 0 (delta 0), pack-reused 576 (from 1)
Receiving objects: 100% (576/576), 82.69 MiB | 27.35 MiB/s, done.


In [ ]:
os.mkdir('datasets')

In [ ]:
shutil.move('car_dataset/dataset/samochody', 'datasets/')
shutil.move('car_dataset/dataset/dataset.yaml', 'datasets/')

'datasets/dataset.yaml'

---

Następnie tworze plik konfiguracyjny zbioru danych

In [ ]:
try:
  with open('datasets/dataset.yaml', "r", encoding="utf-8") as c:
    print(c.read())
except FileNotFoundError:
  print("Nie znaleziono pliku")

path: samochody/
train: images/train/
val: images/test/
nc: 4
names: ['autobus','ciężarowy', 'dostawczy', 'osobowy']  # Nazwy klas


Wczytuje model ze wstępnie zainicjowanymi wagami. Pre-trainowany model był uczony na zbiorze [COCO](https://cocodataset.org/)

In [ ]:
try:
  with open('yolov5/models/yolov5s.yaml', "r", encoding="utf-8") as y:
    print(y.read())
except FileNotFoundError:
  print("Plik nie został znaleziony")

model = YOLO("yolov5su.pt")


In [ ]:
model.info()

YOLOv5s summary: 153 layers, 9,153,152 parameters, 0 gradients, 24.2 GFLOPs


(153, 9153152, 0, 24.2129408)

Tworze plik konfiguracyjny trenowania

In [ ]:
config = {
    "data": "datasets/dataset.yaml",
    "mode": "train",
    "epochs": 600,
    "batch": 16,
    "patience": 0,
    "imgsz": 640,
    # transfer learning
    "freeze": 10, # zamrazam wszystkie warstwy w backbone
    "lr0": 0.001,
    # augmentacja
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "degrees": 30,
    "translate": 0.1,
    "shear": 0.2,
    "flipud": 0.5,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "mixup": 0.2,
    "scale": 0.5
}

In [ ]:
try:
    with open('datasets/cfg.yaml', 'w', encoding='utf-8') as f:
        yc = '\n'.join([f"{key}: {value}" for key, value in config.items()])
        f.write(yc)
except (FileNotFoundError, IsADirectoryError) as e:
    print(f"Błąd zapisu pliku: {e}")

In [ ]:
try:
  with open('datasets/cfg.yaml', "r", encoding="utf-8") as c:
    print(c.read())
except FileNotFoundError:
  print("Nie znaleziono pliku")

data: datasets/dataset.yaml
mode: train
epochs: 600
batch: 16
patience: 0
imgsz: 640
freeze: 10
lr0: 0.001
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 30
translate: 0.1
shear: 0.2
flipud: 0.5
fliplr: 0.5
mosaic: 1.0
mixup: 0.2
scale: 0.5


Następnie przechodze do trenowania modelu

In [ ]:
results = model.train(cfg="datasets/cfg.yaml")

Zapisuje model

In [ ]:
model.save("moj_yolov5_600_w.pt")

Zipuje katalog runs

In [ ]:
folder_to_zip = '/content/runs'

output_zip_file = '/content/runs.zip'

try:
  shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)
except FileNotFoundError:
  print("Plik nie został znaleziony")


print(f"Folder {folder_to_zip} został spakowany do {output_zip_file}")

Folder /content/runs został spakowany do /content/runs.zip


---